Xinference is a powerful tool that supports function calling and is fully compatible with OpenAI's Tool Call API. It allows you to seamlessly integrate and utilize the functionality of OpenAI's tools within your own projects.

# Preparation

First, you need to install Xinference:
```shell
pip install xinference
```

Currently, only ChatGLM3 model is available for tool calls. Here, we install a chatglm cpp for fast inference on Mac. If you are not using a Mac, please follow the install instruction: https://github.com/li-plus/chatglm.cpp#python-binding

```shell
CMAKE_ARGS="-DGGML_METAL=ON" pip install -U chatglm-cpp --no-cache-dir
```

Then, start the Xinference server by the following command:
```shell
xinference
```

The Xinference server will be started:

```shell
2023-11-02 16:04:55,278 xinference   38878 INFO     Xinference successfully started. Endpoint: http://127.0.0.1:9997
2023-11-02 16:04:55,280 xinference.core.supervisor 38878 INFO     Worker 127.0.0.1:32187 has been added successfully
2023-11-02 16:04:55,281 xinference.deploy.worker 38878 INFO     Xinference worker successfully started.
```

Finally, we launch a ChatGLM3 model for tool calls.
```shell
xinference launch -u my_tool_model -n chatglm3 -f ggmlv3 -q q4_0
```



# Function calling

This is an exciting game that showcases the function calling feature. Our objective is to call the AI and obtain a set of lottery numbers to determine if we are lucky enough to win the lottery. To get started, we first need to define a simple function for generating lottery numbers.

In [ ]:
import random

list_red = list(range(1, 34))
list_blue = list(range(1, 17))


def get_lucky_lottery(num):
    total = []
    for _ in range(num):
        res_red = random.sample(list_red, 6)
        res_blue = random.sample(list_blue, 1)
        total.append(res_red + res_blue)
    return total

Then, we describe two functions as a dict, more details refers to https://platform.openai.com/docs/api-reference/chat/create#chat-create-tools

In [ ]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_lucky_lottery",
            "description": "生成双色球彩票号码",
            "parameters": {
                "type": "int",
                "properties": {"num": {"description": "生成的彩票号码组数"}},
                "required": ["num"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "lottery_draw",
            "description": "开奖双色球",
            "parameters": {
                "type": "object",
                "properties": {},
            },
        },
    },
]

Define a function to call above two functions according to the response.

In [ ]:
import functools
import json

FUNCTION_TABLE = {
    "get_lucky_lottery": get_lucky_lottery,
    "lottery_draw": functools.partial(get_lucky_lottery, 1),
}


def handle_response(completion):
    assert "tool_calls" == completion.choices[0].finish_reason
    func_name = completion.choices[0].message.tool_calls[0].function.name
    func_args = completion.choices[0].message.tool_calls[0].function.arguments
    func_kwargs = json.loads(func_args)
    return FUNCTION_TABLE.get(func_name)(**func_kwargs)

Finally, let's ask the LLM to generate 5 group lottery numbers and check if we win the game.

In [ ]:
import openai

client = openai.Client(api_key="not empty", base_url=f"http://127.0.0.1:9997/v1")
completion = client.chat.completions.create(
    model="my_tool_model",
    messages=[{"role": "user", "content": "帮我生成5组双色球号码"}],
    tools=tools,
)
print(f"Lottery numbers: {handle_response(completion)}")
completion = client.chat.completions.create(
    model="my_tool_model", messages=[{"role": "user", "content": "开奖"}], tools=tools
)
print(f"Lottery draw: {handle_response(completion)}")

No lucky, I lose the game.